## Stable Diffusion Web UI Forge

**Homepage :** [SD Web UI Complete setup](https://github.com/ffxvs/sd-webui-complete-setup)  
**Guide :** [Runpod Guide](https://github.com/ffxvs/sd-webui-complete-setup/wiki/Runpod-Guide)

## 1. Requirements

### 1.1. Initialization
**REQUIRED EVERY TIME YOU START THE POD**

In [ ]:
import os
os.chdir('/')
from internal.main import *

initialization_forge()

## 2. Setup Web UI

### 2.1. Extensions
Since the new version of forge, almost extensions are broken.  
If you find any broken extensions, please report them to me on GitHub.

#### • Built-in Extensions
**To install/update these extensions, run the `Extension List` cell below.**

* [Aspect Ratio and Resolution Buttons](https://github.com/altoiddealer/--sd-webui-ar-plusplus)
* [img2img Hires Fix](https://github.com/Amadeus-AI/img2img-hires-fix)
* [Infinite Image Browsing](https://github.com/zanllp/sd-webui-infinite-image-browsing)
* [SD Delete Button](https://github.com/reforget-id/sd_delete_button)
* [State](https://github.com/ilian6806/stable-diffusion-webui-state)
* [Ultimate SD Upscale](https://github.com/Coyote-A/ultimate-upscale-for-automatic1111)

#### • Extension List

In [ ]:
# Run this cell to display the extension list

extensions_selection(builtin_exts_forge_url, extensions_forge_url)
os.chdir(webui)

#### • Install from URLs

In [ ]:
# UPDATE EXTENSIONS
update_exts = boolean[0]

# Install extensions from URL
other_exts = []


################################################################################################################

install_other_exts(other_exts, update_exts)
os.chdir(webui)
completed_message()

## 3. Launch Web UI
Before running this cell, download models using `sd15_resource_lists.ipynb` or `sdxl_resource_lists.ipynb`.  
Run this cell and wait until `Startup time:  *.*s ...` appears.  
Then, click link `https://xxxxx-3001.proxy.runpod.net` or ngrok link `https://xxxxx.ngrok-free.app` to open Web UI.  
Always watch the output log to track image generation progress or errors. 

In [ ]:
# Dark theme
dark_theme = boolean[1]

# Authentication (Recommended for security purposes)
username = ''
password = ''

# Ngrok (Optional)
ngrok_token = ''
ngrok_domain = ''

# CORS (Optional) 
# separate with commas
cors = 'https://huchenlei.github.io'


################################################################################################################
close_port(runpod_port)
launch_webui(dark_theme, username, password, ngrok_token, ngrok_domain, cors)